# FEDS vs. NIFC overlap per pyrome 

## Data sources

In [ ]:
import geopandas as gpd 
# gpd.show_versions()

### FEDS v2 perimeters

Generated with [FEDS v2](https://github.com/Earth-Information-System/fireatlas/tree/conus-dps). [FEDS-PEC](https://github.com/ksharonin/feds-benchmarking) grabs these from the OGC API [(example from VEDA docs)](https://nasa-impact.github.io/veda-docs/notebooks/tutorials/mapping-fires.html). 

### FEDS v3 perimeters 

Generated July-August 2024 using [FEDS v3](https://github.com/Earth-Information-System/fireatlas/tree/primarykeyv2) using settings as defined in the following `.env` file, which should be comparable to those used in v2 runs. 

```
FEDS_FTYPE_OPT="CA"
FEDS_CONT_OPT="CA"
FEDS_EPSG_CODE=9311
FEDS_FIRE_NRT="False"
FEDS_FIRE_SOURCE="SNPP"
FEDS_remove_static_sources="True"
```

FEDS v3 was run in increments like so: `python fireatlas/FireRunDaskCoordinator.py --regnm="ArchiveCONUS2019" --bbox="[-126.401171875,24.071240929282325,-61.36210937500001,49.40003415463647]" --tst="[2019,1,1,\"AM\"]" --ted="[2019,2,1,\"PM\"]"`

To coordinate work, I used the `manual-v3.yaml` (workflow)[https://github.com/Earth-Information-System/fireatlas/blob/2878c0df1b5f031eddbf9f3d4ab870930238b9b4/.github/workflows/manual-v3.yaml] to submit jobs in increments of 3 months each to the MAAP DPS batch computing environment. Example json params for manual v3 workflow: `{"regnm": "ArchiveCONUS2019", "bbox": "[-126.401171875,24.071240929282325,-61.36210937500001,49.40003415463647]", "tst": "[2019,1,1,\"AM\"]", "ted": "[2019,4,1,\"PM\"]", "operation": "--coordinate-all"}`. This operation cannot be temporally parallelized, so it is important that the previous job has finished before the next increment job is submitted. This allows the second job to find and read in the allfires and allpixels files saved by the first job, picking up from where that job left off. The only reason that jobs needed to be split into increments was that if I run a whole year at once, they hit the 24 hour job limit configured in DPS and are killed. 

In [ ]:
feds_v3_filepath = "/projects/shared-buckets/gsfc_landslides/FEDSoutput-v3/ArchiveCONUS2019/2019/allfires_20191231_PM.parq"

### NIFC perimeters

EDIT: trying Historic GeoMAC instead. 

Source: [NIFC InterAgencyFirePerimeterHistory](https://data-nifc.opendata.arcgis.com/datasets/nifc::interagencyfireperimeterhistory-all-years-view/about). 

Downloaded and available in `./data`

In [ ]:
# %%prun -l .25
# nifc = gpd.read_file('./data/Historic_GeoMAC_Perimeters_2019.geojson')
# nifc

In [ ]:
# len(nifc[nifc.fireyear == 2019])

In [ ]:
# sum(nifc[nifc.fireyear == 2019].gisacres)

@TODO Concern: this seems high? 23,095,165 is an order of magnitude larger than NIFC annual acre totals most years, including 2019. What unit is this in? Bc that would explain it. 

### Pyrome boundaries 

Source: Short, Karen C.; Grenfell, Isaac C.; Riley, Karin L.; Vogler, Kevin C. 2020. Pyromes of the conterminous United States. Fort Collins, CO: Forest Service Research Data Archive. https://doi.org/10.2737/RDS-2020-0020

Downloaded and available in `./data`

In [ ]:
# from matplotlib import pyplot as plt
# import matplotlib.image

# img = matplotlib.image.imread('./data/pyrome_boundaries/Pyromes_CONUS_20200206.png')
# imgplot = plt.imshow(img)
# plt.axis('off')
# plt.show()

In [ ]:
pyromes = gpd.read_file('./data/pyrome_boundaries/Pyromes_CONUS_20200206.shp').set_index('PYROME')

In [ ]:
# pyromes

## Setup

In [ ]:
import geopandas as gpd 
# gpd.show_versions()
feds_v3_filepath = "/projects/shared-buckets/gsfc_landslides/FEDSoutput-v3/ArchiveCONUS2019/2019/allfires_20191231_PM.parq"

In [ ]:
import Utilities
from Input_FEDS import InputFEDS
from Input_Reference import InputReference
from Output_Calculation import OutputCalculation 

In [ ]:
# Start time 
year_start = 2019
month_start = 7
day_start = 1 
hour_start = 0
minute_start = 0
second_start = 0
tz_offset_hours_start = 0
tz_offset_minutes_start = 0
utc_offset_start = '00:00'

# End time 
year_stop = 2019
# month_stop = 12
# day_stop = 31
month_stop = 7
day_stop = 2
hour_stop = 0
minute_stop = 0
second_stop = 0
tz_offset_hours_stop = 0
tz_offset_minutes_stop = 0
utc_offset_stop = '00:00'

crs = 4326

# BBOX FOR SEARCH - [lon, lat, lon, lat]
search_bbox = ["-126.401171875","24.071240929282325","-61.36210937500001","49.40003415463647"]

# DAY SEARCH RANGE- acceptable distance warning to search from feds -> reference (e.g. if reference polygon is 8 days away, output a warning)
# note by default, any results not of the same year are eliminated
day_search_range = 7 


calc_in_parallel = False

In [ ]:
# FEDS INPUT SETTINGS

# read local
feds_title = None 
feds_collection = feds_v3_filepath
feds_access_type = "local"
feds_limit = None
feds_filter = False
feds_apply_finalfire = True # only take latest fireID per fire


In [ ]:
# REFERENCE INPUT SETTINGS 
ref_title ="Historic_GeoMAC_Perimeters_2019"
ref_control_type = "defined" # or "custom"
ref_custom_url = "none"
ref_custom_read_type = "none"
ref_filter = False # False or a valid query

In [ ]:
# OUTPUT SETTINGS
print_on = True
plot_on = True
name_for_output_file = "test_run" 
output_format="csv" 
user_path = f"/projects/my-private-bucket/FEDS-PEC"
output_maap_url = f"{user_path}/{name_for_output_file}.{output_format}"

#### Argument processing 

In [ ]:
# start date formatting
search_start = Utilities.format_datetime(year_start, 
                                         month_start, 
                                         day_start, 
                                         hour_start, 
                                         minute_start, 
                                         second_start, 
                                         tz_offset_hours_start, 
                                         tz_offset_minutes_start,
                                         utc_offset_start)
# stop date formatting
search_stop = Utilities.format_datetime(year_stop, 
                                        month_stop, 
                                        day_stop, 
                                        hour_stop, 
                                        minute_stop, 
                                        second_stop, 
                                        tz_offset_hours_stop, 
                                        tz_offset_minutes_stop,
                                        utc_offset_stop)

# bound check the bbox
assert Utilities.check_bbox(search_bbox), f"ERR: passed bbox {search_bbox} is not valid; check bounds"
assert  Utilities.check_crs(crs), f"ERR: invalid crs provided {crs}; please enter valid ESPG CRS number"

### Run calculations

In [ ]:
%%time 
feds_firenrt = InputFEDS(
                 feds_title, 
                 feds_collection, 
                 search_start,
                 search_stop,
                 search_bbox,
                 crs,
                 feds_access_type,
                 feds_limit,
                 feds_filter,
                 feds_apply_finalfire
                )

feds_firenrt

In [ ]:
%%time

nifc_search = InputReference( 
                 search_start,
                 search_stop,
                 search_bbox,
                 crs,
                 ref_title,
                 ref_control_type,
                 ref_custom_url,
                 ref_custom_read_type,
                 ref_filter,
                )

In [ ]:
import sys

sys.getsizeof(nifc_search.polygons)

In [ ]:
%%time

my_output = OutputCalculation(
                feds_firenrt,
                nifc_search,
                output_format, 
                output_maap_url,
                day_search_range,
                print_on,
                plot_on, 
                calc_in_parallel
                )

## Analysis

### Per-pyrome analysis


#### Summary stats over CONUS

So: are we only looking at matches? 

Yes. Several reasons:
- If a FEDS perim does not match a NIFC perim, it is more likely that there isn't a matching fire in NIFC than that there is a perim for the same incident, but they are totally non-overlapping.
- i would expect that large fires will intersect somewhat spatially in almost all cases.

What this does not tell us about: 
- of all NIFC perimeters, how many have FEDS matches? (If FEDS misses a lot of large NIFC fires, that is a problem)
- of all FEDS perimeters, how many have NIFC matches? (we expect that many FEDS fires will not be in NIFC- this could be due to them actually being false positives or to the fact that many actual fires are not captured in the NIFC base)

When they are looking at similar things, how well do they agree? 

How can we learn where the perimeters are flawed so we can build that understanding into future uses? 

For known matches are we seeing systematic errors in different pyromes? 

Summary stats: 

avg ratio, accuracy, precision, recall, iou, f1, symm_ratio. should just be column means. 

In [ ]:
%%time

import pandas as pd

results = gpd.read_file("/projects/my-private-bucket/FEDS-PEC/ArchiveCONUS2019-julyaugustsept.csv")
results['ref_polygon'] = gpd.GeoSeries.from_wkt(results['ref_polygon'])
results['feds_polygon'] = gpd.GeoSeries.from_wkt(results['feds_polygon'])

print("-------------------") 
print("CONUS column mean values:")
for stat in ['ratio', 'accuracy', 'recall', 'iou', 'f1', 'symm_ratio']:
    results[stat] = pd.to_numeric(results[stat])
    print(f"{stat} mean: {results[stat].mean():.3f}")

print("-------------------\n")

# pyromes = pyromes.to_crs(feds_firenrt.crs)
pyromes = pyromes.to_crs(epsg=4326) 

stats = gpd.GeoDataFrame(columns=['pyrome_name', 'ratio', 'accuracy', 'recall', 'iou', 'f1', 'symm_ratio']).set_index('pyrome_name')

for index, pyrome in pyromes.iterrows():
    print(f"{pyrome.NAME}:")

    # idea: # select rows where ref_polyon is half or more within pyrome geometry
    # current: select any row where ref_polygolkjlkjn intersects pyrome (possibility of duplication along boundaries) 
    subset = results[results['ref_polygon'].intersects(pyrome.geometry)] 

    print(f"\t{len(subset)} matches")
    
    if len(subset) < 1:
        pass
    else: 
        for stat in ['ratio', 'accuracy', 'recall', 'iou', 'f1', 'symm_ratio']:
            print(f"\t{stat} mean: {subset[stat].mean():.3f}")
            stats.at[pyrome.NAME, stat] = subset[stat].mean()


outpath = f"{user_path}/{name_for_output_file}_pyrome_stats.csv"
# stats.to_csv(outpath)

# add row with nan vals for all pyromes that dont have any matches

In [ ]:
jun = gpd.read_file("~/my-private-bucket/FEDS-PEC/ArchiveCONUS2019-jun.csv")
july = gpd.read_file("~/my-private-bucket/FEDS-PEC/ArchiveCONUS2019-julyaugustsept.csv")

In [ ]:
print(len(jun))
print(len(july))


In [ ]:
jun.columns

In [ ]:
combined = pd.concat([jun, july])

In [ ]:
type(combined)

In [ ]:
print(len(combined))
len(combined) == len(jun) + len(july)

In [ ]:
combined.to_csv("~/my-public-bucket/ArchiveCONUS2019-nifc-matches-june-sept.csv")

## Plotting 

In [ ]:
import geopandas as gpd 
import pandas as pd
import folium

df = pd.read_csv("~/my-public-bucket/ArchiveCONUS2019-nifc-matches-june-sept.csv")
df = df.drop(["Unnamed: 0", "geometry"], axis=1)
gdf = gpd.GeoDataFrame(df)

gdf['ref_polygon'] = gpd.GeoSeries.from_wkt(gdf['ref_polygon']).set_crs(epsg=4326)
gdf['feds_polygon'] = gpd.GeoSeries.from_wkt(gdf['feds_polygon']).set_crs(epsg=4326)
gdf = gdf.set_geometry('feds_polygon')

ref_polygon = gdf[['ref_polygon', 'incident_name', 'ref_timestamp']].set_geometry('ref_polygon')
feds_polygon = gdf[['feds_polygon', 'incident_name', 'feds_timestamp']].set_geometry('feds_polygon') 

In [ ]:
pyromes = gpd.read_file('./data/pyrome_boundaries/Pyromes_CONUS_20200206.shp').set_index('PYROME')
pyromes = pyromes.to_crs(epsg=4326) 

In [ ]:
import Utilities
from Input_Reference import InputReference

# Start time 
year_start = 2019
month_start = 6
day_start = 1 
hour_start = 0
minute_start = 0
second_start = 0
tz_offset_hours_start = 0
tz_offset_minutes_start = 0
utc_offset_start = '00:00'

# End time 
year_stop = 2019
# month_stop = 12
# day_stop = 31
month_stop = 10
day_stop = 1
hour_stop = 0
minute_stop = 0
second_stop = 0
tz_offset_hours_stop = 0
tz_offset_minutes_stop = 0
utc_offset_stop = '00:00'

crs = 4326

# BBOX FOR SEARCH - [lon, lat, lon, lat]
search_bbox = ["-126.401171875","24.071240929282325","-61.36210937500001","49.40003415463647"]

# DAY SEARCH RANGE- acceptable distance warning to search from feds -> reference (e.g. if reference polygon is 8 days away, output a warning)
# note by default, any results not of the same year are eliminated
day_search_range = 7 

# REFERENCE INPUT SETTINGS 
ref_title ="Historic_GeoMAC_Perimeters_2019"
ref_control_type = "defined" # or "custom"
ref_custom_url = "none"
ref_custom_read_type = "none"
ref_filter = False # False or a valid query

# start date formatting
search_start = Utilities.format_datetime(year_start, 
                                         month_start, 
                                         day_start, 
                                         hour_start, 
                                         minute_start, 
                                         second_start, 
                                         tz_offset_hours_start, 
                                         tz_offset_minutes_start,
                                         utc_offset_start)
# stop date formatting
search_stop = Utilities.format_datetime(year_stop, 
                                        month_stop, 
                                        day_stop, 
                                        hour_stop, 
                                        minute_stop, 
                                        second_stop, 
                                        tz_offset_hours_stop, 
                                        tz_offset_minutes_stop,
                                        utc_offset_stop)

# bound check the bbox
assert Utilities.check_bbox(search_bbox), f"ERR: passed bbox {search_bbox} is not valid; check bounds"
assert  Utilities.check_crs(crs), f"ERR: invalid crs provided {crs}; please enter valid ESPG CRS number"

nifc_search = InputReference( 
                 search_start,
                 search_stop,
                 search_bbox,
                 crs,
                 ref_title,
                 ref_control_type,
                 ref_custom_url,
                 ref_custom_read_type,
                 ref_filter,
                )

# Was getting some NIFC matches outside of date range, so re-filtering here (noticed due to Kincade fire) 

start = pd.to_datetime(search_start).to_datetime64()
stop = pd.to_datetime(search_stop).to_datetime64()

filtered_nifc = nifc_search.polygons[(nifc_search.polygons.DATE_CUR_STAMP >= start) & (nifc_search.polygons.DATE_CUR_STAMP <= stop)]

filtered_nifc.DATE_CUR_STAMP = filtered_nifc.DATE_CUR_STAMP.astype(str)

In [ ]:
# start = pd.to_datetime(search_start)
# stop = pd.to_datetime(search_stop)
print(f"pre-filtering ref_polygon: {len(ref_polygon)}") 
ref_polygon = ref_polygon[(pd.to_datetime(ref_polygon.ref_timestamp) >= start) & (pd.to_datetime(ref_polygon.ref_timestamp) <= stop)]
print(f"filtered: {len(ref_polygon)}")

In [ ]:
# m = folium.Map(center=[39.5, -119.5], zoom_start=15, min_zoom=10, max_bounds=True)
m = folium.Map()

esri_world_terrain = folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}',
    attr='Tiles &copy; Esri &mdash; Source: USGS, Esri, TANA, DeLorme, and NPS',
    name='Esri WorldTerrain',
    overlay=False,
    control=False
)

esri_world_terrain.add_to(m)


# @TODO need to filter down to only the latest perimeter from a given incident NIFC
# groupby incident name, take latest should work OR just have fill opacity 1- then you cnt see. 
# @TODO could add popups with correct info; or, could just not. 

pyromes.explore(m=m, style_kwds={'fillOpacity': 0, 'weight': .75}, highlight_kwds={'fillOpacity':0}, color='black', name='Pyrome boundaries')

filtered_nifc.explore(m=m, style_kwds={'fillOpacity': 0}, color='red', name='All NIFC perimeters (red)')

ref_polygon.explore(m=m, style_kwds={'fillOpacity': .5}, color='blue', name='NIFC matched perimeters (blue)')

feds_polygon.explore(m=m, style_kwds={'fillOpacity': .5}, color='orange', name='FEDS matched perimeters (orange)') 

folium.LayerControl(collapsed=False).add_to(m)

m

In [ ]:
m.save("summer-2019-FEDS-vs-NIFC-by-pyrome.html")